In [1]:
import os, openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

os.environ["LANGCHAIN_PROJECT"] = "financial-agent"

## Setup tools

In [2]:
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.tools import PolygonLastQuote, PolygonTickerNews, PolygonFinancials, PolygonAggregates

polygon = PolygonAPIWrapper()
tools = [
    PolygonLastQuote(api_wrapper=polygon),
    PolygonTickerNews(api_wrapper=polygon),
    PolygonFinancials(api_wrapper=polygon),
    PolygonAggregates(api_wrapper=polygon)
]

In [3]:
for tool in tools:
    print(tool.name)

polygon_last_quote
polygon_ticker_news
polygon_financials
polygon_aggregates


In [4]:
from langchain_openai.chat_models import AzureChatOpenAI
llm  = AzureChatOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment="gpt-4o",
    streaming=True,
    temperature=0
)

In [5]:
llm_with_tools = llm.bind_tools(tools)

query = "Why is CRM stock down so much? Was something wrong with its earnings? Could you be specific what went wrong."
result = llm_with_tools.invoke(query)

result

AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_3sQoTPRy99GOjTdfaqwIxFmc', 'function': {'arguments': '{"query": "CRM"}', 'name': 'polygon_ticker_news'}, 'type': 'function'}, {'index': 1, 'id': 'call_Wtc58fRR4gJbAj1kAkOBaFWV', 'function': {'arguments': '{"query": "CRM"}', 'name': 'polygon_financials'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-391bb067-d7a3-40be-bf48-d6d9f1c49aab-0', tool_calls=[{'name': 'polygon_ticker_news', 'args': {'query': 'CRM'}, 'id': 'call_3sQoTPRy99GOjTdfaqwIxFmc'}, {'name': 'polygon_financials', 'args': {'query': 'CRM'}, 'id': 'call_Wtc58fRR4gJbAj1kAkOBaFWV'}])

In [6]:
# A tool can have multiple functions schemas inside it that can be used to process the input data.

result.tool_calls

[{'name': 'polygon_ticker_news',
  'args': {'query': 'CRM'},
  'id': 'call_3sQoTPRy99GOjTdfaqwIxFmc'},
 {'name': 'polygon_financials',
  'args': {'query': 'CRM'},
  'id': 'call_Wtc58fRR4gJbAj1kAkOBaFWV'}]

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

tool_mapping = {
    "polygon_last_quote": tools[0],
    "polygon_ticker_news": tools[1],
    "polygon_financials": tools[2],
    "polygon_aggregates": tools[3]
}

for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [8]:
for message in messages:
    print(message.type)

human
ai
tool
tool



## Agent

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        """
        As a Financial Research Assistant, your role is to provide latest financial insights about companies and help individuals make informed investment decisions. 
        
        You have access to 4 tools from Polygon.io API that lets you query the latest financial data from all US stock exchanges. The tools are:
        1. polygon_last_quote: Get the latest quote for ticker
        2. polygon_ticker_news: Get the latest news for ticker
        3. polygon_financials: Get the financials for ticker
        4. polygon_aggregates: Get the aggregates for ticker   

        You may need one or more tools to answer a user query. You can use the tools in any order depending on the user query. 
        
        Conversaion History:
        {chat_history}
        """                 
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "User Input: {input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

prompt.pretty_print()

================================ System Message ================================


        As a Financial Research Assistant, your role is to provide latest financial insights about companies and help individuals make informed investment decisions. 
        
        You have access to 4 tools from Polygon.io API that lets you query the latest financial data from all US stock exchanges. The tools are:
        1. polygon_last_quote: Get the latest quote for ticker
        2. polygon_ticker_news: Get the latest news for ticker
        3. polygon_financials: Get the financials for ticker
        4. polygon_aggregates: Get the aggregates for ticker   

        You may need one or more tools to answer a user query. You can use the tools in any order depending on the user query. 
        
        Conversaion History:
        {chat_history}
        

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message 

LangChain has several abstractions to make working with agents easy.

### AgentAction
This is a dataclass that represents the action an agent should take. It has a tool property (which is the name of the tool that should be invoked) and a tool_input property (the input to that tool)

### AgentFinish
This represents the final result from an agent, when it is ready to return to the user. It contains a return_values key-value mapping, which contains the final agent output. Usually, this contains an output key containing a string that is the agent's response.

### Intermediate Steps
These represent previous agent actions and corresponding outputs from this CURRENT agent run. These are important to pass to future iteration so the agent knows what work it has already done. This is typed as a List[Tuple[AgentAction, Any]]. Note that observation is currently left as type Any to be maximally flexible. In practice, this is often a string.

### `AgentExecutor` is a built-in class that provides the `run_agent` functionality

```console
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "retrieve": retrieve, 
            "calculate_total": calculate_total
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

```

Also adds additional functionalities such as logging, error handling for tools and also the entire agent.

In [10]:
import random
from langchain_community.chat_message_histories.cosmos_db import CosmosDBChatMessageHistory

cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
    cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
    cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
    connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
    session_id="shiva-test" + str(random.randint(1, 10000)),
    user_id="shivac"
    )
cosmos.prepare_cosmos()

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(k=10, return_messages=True,memory_key="chat_history",chat_memory=cosmos)

In [11]:
from langchain.agents import create_openai_functions_agent, AgentExecutor

agent_runnable = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent_runnable, tools=tools, verbose=False, memory=memory) 
# verbose=True to see the agent's internal state i.e. which tools are being used by the agent to answer the user query

In [12]:
result = agent_executor.invoke({"input": "How can you help me?"}) 

In [13]:
result = agent_executor.invoke({"input": "Why is CRM stock down so much? Was something wrong with its earnings? Could you be specific what went wrong."})

In [14]:
print(result["output"])

Based on the latest financial data and news articles, here are some insights into why Salesforce (CRM) stock might be down and what went wrong with its earnings:

### Financial Performance
1. **Revenue Growth**: Salesforce reported revenues of $34.86 billion for the fiscal year ending January 31, 2024, and $9.29 billion for Q4 2024. While these figures show growth, the rate of growth might not have met investor expectations.
2. **Net Income**: The net income for the fiscal year was $4.14 billion, with a net income of $1.45 billion for Q4 2024. Although profitable, the earnings per share (EPS) of $4.20 (diluted) for the fiscal year and $1.47 for Q4 might have been below market expectations.
3. **Operating Expenses**: High operating expenses, including significant costs in research and development ($4.91 billion for the fiscal year), could have impacted profitability.
4. **Cash Flow**: The net cash flow from operating activities was strong at $10.23 billion for the fiscal year, but the n

In [ ]:
# # Chatbot

# import time

# print("I am a Financial Research Assistant! How can I assist you?\n")
# while True:
#     query = input()
#     if query in ["quit", "exit"]:
#         break
#     start_time = time.time()
#     result = agent_executor.invoke({"input": query})
#     end_time = time.time()
#     execution_time = end_time - start_time
#     print("\nUser: {0}".format(query))
#     print("\nAssistant: {0}".format(result['output']))
#     print(f"\tTime taken to respond: {round(execution_time)} seconds")

## Gradio Interface

In [25]:
import gradio as gr

def run_agent(query):
    result = agent_executor.invoke({"input": query})
    return result['output']

ui = gr.Interface(
    fn=run_agent,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=gr.Markdown(),
    title="Financial Agent",
    description="Financial Data Explorer: Leveraging Advanced API Tools for Market Insights"
)

ui.launch()

c:\Users\shchitt\AppData\Local\anaconda3\envs\langgraph\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
